## 1. Create the compute instance

In [ ]:
# import required libraries
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import WorkspaceConnection
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

from azure.ai.ml.entities import ComputeInstance 
from azure.ai.ml.entities import CustomApplications, ImageSettings, EndpointsSettings, VolumeSettings 

ml_client = MLClient.from_config(credential=credential)

In [ ]:
# create custom application from default promptflow image
image = ImageSettings(reference='mcr.microsoft.com/azureml/promptflow/promptflow-runtime-stable:latest') 
endpoints = [EndpointsSettings(published=8081, target=8080)]
app = CustomApplications(name='promptflow-runtime',endpoints=endpoints,bind_mounts=[],image=image,environment_variables={}) 

In [ ]:
# create compute instance for prompt flow
ci_basic_name = "contoso-chat-ci"
ci_basic = ComputeInstance(name=ci_basic_name, size="STANDARD_DS12_V2",custom_applications=[app]) 
ml_client.begin_create_or_update(ci_basic)

In [ ]:
# List compute instances
for ci in ml_client.compute.list():
    print(ci.name)

## 2. Create the prompt flow runtime

Once the compute instance is created. [Follow these instructions to create a runtime in AI Studio.](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/create-manage-runtime)

## 3. Run the prompt flow on the runtime to install the dependencies

In [ ]:
from promptflow import PFClient
pf_client = PFClient()

In [ ]:
# Add a question to test the base prompt flow.
output = pf_client.test(
    flow=".", # Path to the flow directory
    inputs={},
)
output['version']

In [ ]:
config_path = "../config.json"
from promptflow.azure import PFClient
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

In [ ]:
# Update the runtime to the name of the runtime you created previously
runtime = "promptflow-contoso-chat-rt"
# load flow
flow = "."
run_name="update_runtime_test"
data = "../data/emptydata.jsonl"

In [ ]:
base_run = pf_azure_client.run(
    flow=flow,
    data=data,
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name,
    name=run_name
)
print(base_run)

In [ ]:
pf_azure_client.stream(base_run)

In [ ]:
details = pf_azure_client.get_details(base_run)

In [ ]:
details["outputs.version"][0]